<a href="https://colab.research.google.com/github/realdanielalexander/diamonds_bangkit/blob/master/Diamonds_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Run this colab on tensorflow 2.x</h1>

In [0]:
#@title Run this Colab on TensorFlow 2.x
%tensorflow_version 2.x

<h1>Import relevant libraries</h1>

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

<h1>Load dataset</h1>

In [0]:
from google.colab import files
uploaded = files.upload()
import io
df2 = pd.read_csv(io.BytesIO(uploaded['diamonds.csv']))
# Dataset is now stored in a Pandas Dataframe

In [13]:
df_raw = pd.read_csv("diamonds.csv")
df_raw

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


## Define functions that build and train a model

This code block is taken from the 'Programming Exercise' module on Machine Learning Crash Course.


In [22]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model               


def train_model(model, df, feature, label, my_epochs, 
                my_batch_size=None, my_validation_split=0.1):
  """Feed a dataset into the model in order to train it."""

  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=my_batch_size,
                      epochs=my_epochs,
                      validation_split=my_validation_split)

  # Gather the model's trained weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the 
  # rest of history.
  epochs = history.epoch
  
  # Isolate the root mean squared error for each epoch.
  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]

  return epochs, rmse, history.history   

print("Defined the build_model and train_model functions.")

Defined the build_model and train_model functions.


## Define plotting functions

This code block is taken from the 'Programming Exercise' module on Machine Learning Crash Course.

In [23]:
#@title Define the plotting function

def plot_the_loss_curve(epochs, mae_training, mae_validation):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs[1:], mae_training[1:], label="Training Loss")
  plt.plot(epochs[1:], mae_validation[1:], label="Validation Loss")
  plt.legend()
  
  # We're not going to plot the first epoch, since the loss on the first epoch
  # is often substantially greater than the loss for other epochs.
  merged_mae_lists = mae_training[1:] + mae_validation[1:]
  highest_loss = max(merged_mae_lists)
  lowest_loss = min(merged_mae_lists)
  delta = highest_loss - lowest_loss
  print(delta)

  top_of_y_axis = highest_loss + (delta * 0.05)
  bottom_of_y_axis = lowest_loss - (delta * 0.05)
   
  plt.ylim([bottom_of_y_axis, top_of_y_axis])
  plt.show()  

print("Defined the plot_the_loss_curve function.")

Defined the plot_the_loss_curve function.


<h1>Plot the correlation matrix</h1>

In [14]:
df.corr()

,Unnamed: 0,carat,depth,table,price,x,y,z
Unnamed: 0,1.000000,-0.377983,-0.034800,-0.100830,-0.306873,-0.405440,-0.395843,-0.399208
carat,-0.377983,1.000000,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
depth,-0.034800,0.028224,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,-0.100830,0.181618,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,-0.306873,0.921591,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,-0.405440,0.975094,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,-0.395843,0.951722,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,-0.399208,0.953387,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


From this correlation matrix, we can see that the biggest correlating variables are carat and price. So we're going to choose carat as a feature and price as the label. We found that correlation between carrat with price and volume (x,y,z) are the highest. But, we decided to only chose 2 features so we chose carat and price.

<h1>Copy features and label to varable df</h1>

In [16]:
df = data[['carat', 'price']].copy()
df

,carat,price
0,0.23,326
1,0.21,326
2,0.23,327
3,0.29,334
4,0.31,335
...,...,...
53935,0.72,2757
53936,0.72,2757
53937,0.70,2757
53938,0.86,2757


We found that correlation between carrat with price and volume (x,y,z) are the highest. But, we decided to only chose 2 features so we chose carrat and price.

<h1>Examine the dataset</h1>

In [17]:
df.describe()

,carat,price
count,53940.000000,53940.000000
mean,0.797940,3932.799722
std,0.474011,3989.439738
min,0.200000,326.000000
25%,0.400000,950.000000
50%,0.700000,2401.000000
75%,1.040000,5324.250000
max,5.010000,18823.000000


Based on the description table, there are some anomalies on the data. The maximum value for both the feature and the label are much greater than its quantiles. There must be outliers in this dataset.

<h1>Splitting into training and testing data</h1>

In [0]:
from sklearn.model_selection import train_test_split
training_df, testing_df = train_test_split(df, test_size=0.2)

In [29]:
training_df.shape

(43152, 2)

In [30]:
testing_df.shape

(10788, 2)

<h1>Building the model</h1>

In [31]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 30
batch_size = 30

# Specify the feature and the label.
my_feature = "carat"  # carat value.
my_label="price" # the price value for that specific diamond.
# We're predicting diamond price based on carat value

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(my_model, training_df, 
                                         my_feature, my_label,
                                         epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

plot_the_model(weight, bias, my_feature, my_label)
plot_the_loss_curve(epochs, rmse)

Epoch 1/30
1295/1295 [==============================] - 2s 1ms/step - loss: 31209242.0000 - root_mean_squared_error: 5586.2290 - val_loss: 32120394.0000 - val_root_mean_squared_error: 5668.4277
Epoch 2/30
1295/1295 [==============================] - 2s 1ms/step - loss: 30985668.0000 - root_mean_squared_error: 5566.5381 - val_loss: 31895530.0000 - val_root_mean_squared_error: 5648.5542
Epoch 3/30
1295/1295 [==============================] - 2s 1ms/step - loss: 30765240.0000 - root_mean_squared_error: 5546.8374 - val_loss: 31671790.0000 - val_root_mean_squared_error: 5628.7104
Epoch 4/30
1295/1295 [==============================] - 2s 1ms/step - loss: 30557994.0000 - root_mean_squared_error: 5527.1099 - val_loss: 31448750.0000 - val_root_mean_squared_error: 5608.8574
Epoch 5/30
1295/1295 [==============================] - 2s 1ms/step - loss: 30332092.0000 - root_mean_squared_error: 5507.3516 - val_loss: 31227128.0000 - val_root_mean_squared_error: 5589.0630
Epoch 6/30
1295/1295 [========

ValueError: ignored